# 📖 TABLE OF CONTENTS

- [1. Introduction]()
- [2. Installing Dependencies]()
- [3. Mount Google Drive & Load API Keys]()
- [4. Role Prompting]()
  - [Example]()
- [5. Few Shot Prompting]()
- [6. Chain Prompting]()
- [7. Bad Prompt Practices]()
  - [1. Bad Prompt Example 1 - Open-ended Prompt]()
  - [2. Bad Prompt Example 2 - Unclear Prompt]()
- [8. Chain of Thought Prompting]()
- [9. Tips for Effective Prompt Engineering]()

![rainbow](https://github.com/ancilcleetus/My-Learning-Journey/assets/25684256/839c3524-2a1d-4779-85a0-83c562e1e5e5)

# 1. Introduction

Prompt engineering is a relatively new discipline that involves developing and optimizing prompts to use language models for various applications and research topics efficiently. It helps to understand the capabilities and limitations of LLMs better and is essential for many NLP tasks. We will provide practical examples to demonstrate the difference between good and bad prompts, helping you to understand the nuances of prompt engineering better.

By the end of this lesson, you will have a solid foundation in the knowledge and strategies needed to create powerful prompts that enable LLMs to deliver accurate, contextually relevant, and insightful responses.

![rainbow](https://github.com/ancilcleetus/My-Learning-Journey/assets/25684256/839c3524-2a1d-4779-85a0-83c562e1e5e5)

# 2. Installing Dependencies

In [1]:
!pip3 install langchain==0.0.208 deeplake openai==0.27.8 python-dotenv tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.9/607.9 kB 7.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 54.6 MB/s eta 0:00:00


![rainbow](https://github.com/ancilcleetus/My-Learning-Journey/assets/25684256/839c3524-2a1d-4779-85a0-83c562e1e5e5)

# 3. Mount Google Drive & Load API Keys

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


All the API Keys are stored in file "llm_env". It's contents are as below:

ACTIVELOOP_TOKEN=<[Your Activeloop API Key](https://app.activeloop.ai/register)>

OPENAI_API_KEY=<[Your OpenAI API Key](https://platform.openai.com/)>

GOOGLE_API_KEY=<[Your Google API Key](https://console.cloud.google.com/apis/credentials)>

GOOGLE_CSE_ID=<[Your Google Custom Search Engine ID](https://programmablesearchengine.google.com/controlpanel/create)>

HUGGINGFACEHUB_API_TOKEN=<[Your Hugging Face Access Token](https://huggingface.co/settings/tokens)>

In [2]:
from dotenv import load_dotenv

# Load API Keys for Deep Lake Vector Database, OpenAI, Google & Hugging Face
load_dotenv('/content/drive/MyDrive/ancilcleetus-github/llm_env')

True

In [ ]:
import os

print(f"os.environ['ACTIVELOOP_TOKEN']: \n{os.environ['ACTIVELOOP_TOKEN']}")
print(f"os.environ['OPENAI_API_KEY']: \n{os.environ['OPENAI_API_KEY']}")
print(f"os.environ['GOOGLE_API_KEY']: \n{os.environ['GOOGLE_API_KEY']}")
print(f"os.environ['GOOGLE_CSE_ID']: \n{os.environ['GOOGLE_CSE_ID']}")
print(f"os.environ['HUGGINGFACEHUB_API_TOKEN']: \n{os.environ['HUGGINGFACEHUB_API_TOKEN']}")

![rainbow](https://github.com/ancilcleetus/My-Learning-Journey/assets/25684256/839c3524-2a1d-4779-85a0-83c562e1e5e5)

# 4. Role Prompting

Role prompting involves asking the LLM to assume a specific role or identity before performing a given task, such as acting as a copywriter. This can help guide the model's response by providing a context or perspective for the task. To work with role prompts, you could iteratively:

1. Specify the role in your prompt, e.g., "As a copywriter, create some attention-grabbing taglines for AWS services."
2. Use the prompt to generate an output from an LLM.
3. Analyze the generated response and, if necessary, refine the prompt for better results.

## Example

In this example, the LLM is asked to act as a futuristic robot band conductor and suggest a song title related to the given theme and year.

In [ ]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import OpenAI

# Initialize LLM
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)

template = """
As a futuristic robot band conductor, I need you to help me come up with a song title.
What's a cool song title for a song about {theme} in the year {year}?
"""
prompt = PromptTemplate(
    input_variables=["theme", "year"],
    template=template,
)

# Input data for the prompt
input_data = {"theme": "interstellar travel", "year": "3030"}

# Create LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

# Run the LLMChain to get the AI-generated song title
response = chain.run(input_data)

print("Theme: interstellar travel")
print("Year: 3030")
print(f"AI-generated song title: {response}")

Theme: interstellar travel
Year: 3030
AI-generated song title: 
"Galactic Odyssey: Journey to 3030"


This is a good prompt for several reasons:

- **Clear instructions:** The prompt is phrased as a clear request for help in generating a song title, and it specifies the context: "As a futuristic robot band conductor." This helps the LLM understand that the desired output should be a song title related to a futuristic scenario.

- **Specificity:** The prompt asks for a song title that relates to a specific theme and a specific year, "{theme} in the year {year}." This provides enough context for the LLM to generate a relevant and creative output. The prompt can be adapted for different themes and years by using input variables, making it versatile and reusable.

- **Open-ended creativity:** The prompt allows for open-ended creativity, as it doesn't limit the LLM to a particular format or style for the song title. The LLM can generate a diverse range of song titles based on the given theme and year.

- **Focus on the task:** The prompt is focused solely on generating a song title, making it easier for the LLM to provide a suitable output without getting sidetracked by unrelated topics.

These elements help the LLM understand the user's intention and generate a suitable response.

![rainbow](https://github.com/ancilcleetus/My-Learning-Journey/assets/25684256/839c3524-2a1d-4779-85a0-83c562e1e5e5)

# 5. Few Shot Prompting

In the next example, the LLM is asked to provide the emotion associated with a given color based on a few examples of color-emotion pairs:

In [ ]:
from langchain import PromptTemplate, FewShotPromptTemplate, LLMChain
from langchain.llms import OpenAI

# Initialize LLM
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)

examples = [
    {"color": "red", "emotion": "passion"},
    {"color": "blue", "emotion": "serenity"},
    {"color": "green", "emotion": "tranquility"},
]

example_formatter_template = """
Color: {color}
Emotion: {emotion}\n
"""
example_prompt = PromptTemplate(
    input_variables=["color", "emotion"],
    template=example_formatter_template,
)

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Here are some examples of colors and the emotions associated with them:\n\n",
    suffix="\n\nNow, given a new color, identify the emotion associated with it:\n\nColor: {input}\nEmotion:",
    input_variables=["input"],
    example_separator="\n",
)

formatted_prompt = few_shot_prompt.format(input="Yellow")

# Create the LLMChain for the prompt
chain = LLMChain(llm=llm, prompt=PromptTemplate(template=formatted_prompt, input_variables=[]))

# Run the LLMChain to get the AI-generated emotion associated with the input color
response = chain.run({})

print("Color: Yellow")
print(f"Emotion: {response}")

Color: Yellow
Emotion:  happiness


This prompt provides clear instructions and few-shot examples to help the model understand the task.

![rainbow](https://github.com/ancilcleetus/My-Learning-Journey/assets/25684256/839c3524-2a1d-4779-85a0-83c562e1e5e5)

# 6. Chain Prompting

Chain Prompting refers to the practice of chaining consecutive prompts, where the output of a previous prompt becomes the input of the successive prompt.

To use chain prompting with LangChain, you could:

- Extract relevant information from the generated response.
- Use the extracted information to create a new prompt that builds upon the previous response.
- Repeat steps as needed until the desired output is achieved.

`PromptTemplate` class makes constructing prompts with dynamic inputs easier. This is useful when creating a prompt chain that depends on previous answers.

In [4]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import OpenAI

# Initialize LLM
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)

# Prompt 1
template_question = """What is the name of the famous scientist who developed the theory of general relativity?
Answer: """
prompt_question = PromptTemplate(template=template_question, input_variables=[])

# Prompt 2
template_fact = """Provide a brief description of {scientist}'s theory of general relativity.
Answer: """
prompt_fact = PromptTemplate(template=template_fact, input_variables=["scientist"])

# Create the LLMChain for the first prompt
chain_question = LLMChain(llm=llm, prompt=prompt_question)

# Run the LLMChain for the first prompt with an empty dictionary
response_question = chain_question.run({})

# Extract the scientist's name from the response
scientist = response_question.strip()

# Create the LLMChain for the second prompt
chain_fact = LLMChain(llm=llm, prompt=prompt_fact)

# Input data for the second prompt
input_data = {"scientist": scientist}

# Run the LLMChain for the second prompt
response_fact = chain_fact.run(input_data)

print(f"Scientist: {scientist}")
print(f"Fact: {response_fact}")

Scientist: Albert Einstein
Fact: 
Albert Einstein's theory of general relativity is a theory of gravity that explains how massive objects interact with each other and how they affect the fabric of space and time. According to this theory, gravity is not a force between masses, but rather a curvature of space and time caused by the presence of massive objects. This curvature is what causes objects to move towards each other, as if they were being pulled by a force. General relativity also predicts that the path of an object in space is affected by the presence of other massive objects, and that the passage of time is affected by the strength of gravity. This theory has been confirmed by numerous experiments and is considered one of the most important and influential theories in modern physics.


![rainbow](https://github.com/ancilcleetus/My-Learning-Journey/assets/25684256/839c3524-2a1d-4779-85a0-83c562e1e5e5)

# 7. Bad Prompt Practices

Now, let's see some examples of prompting that are generally considered bad.

## 1. Bad Prompt Example 1 - Open-ended Prompt

This prompt may generate a less informative or focused response than the previous chain prompt example due to its more open-ended nature.

In [5]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import OpenAI

# Initialize LLM
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)

# Prompt 1
template_question = """What is the name of the famous scientist who developed the theory of general relativity?
Answer: """
prompt_question = PromptTemplate(template=template_question, input_variables=[])

# Prompt 2
template_fact = """Tell me something interesting about {scientist}.
Answer: """
prompt_fact = PromptTemplate(template=template_fact, input_variables=["scientist"])

# Create the LLMChain for the first prompt
chain_question = LLMChain(llm=llm, prompt=prompt_question)

# Run the LLMChain for the first prompt with an empty dictionary
response_question = chain_question.run({})

# Extract the scientist's name from the response
scientist = response_question.strip()

# Create the LLMChain for the second prompt
chain_fact = LLMChain(llm=llm, prompt=prompt_fact)

# Input data for the second prompt
input_data = {"scientist": scientist}

# Run the LLMChain for the second prompt
response_fact = chain_fact.run(input_data)

print(f"Scientist: {scientist}")
print(f"Fact: {response_fact}")

Scientist: Albert Einstein
Fact: 
Albert Einstein was a passionate musician and was known to play the violin daily. He even said that if he hadn't become a scientist, he would have become a musician. He also believed that music helped him think and come up with new ideas.


## 2. Bad Prompt Example 2 - Unclear Prompt

In [6]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import OpenAI

# Initialize LLM
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)

# Prompt 1
template_question = """What are some musical genres?
Answer: """
prompt_question = PromptTemplate(template=template_question, input_variables=[])

# Prompt 2
template_fact = """Tell me something about {genre1}, {genre2}, and {genre3} without giving any specific details.
Answer: """
prompt_fact = PromptTemplate(input_variables=["genre1", "genre2", "genre3"], template=template_fact)

# Create the LLMChain for the first prompt
chain_question = LLMChain(llm=llm, prompt=prompt_question)

# Run the LLMChain for the first prompt with an empty dictionary
response_question = chain_question.run({})

# Assign three hardcoded genres
genre1, genre2, genre3 = "jazz", "pop", "rock"

# Create the LLMChain for the second prompt
chain_fact = LLMChain(llm=llm, prompt=prompt_fact)

# Input data for the second prompt
input_data = {"genre1": genre1, "genre2": genre2, "genre3": genre3}

# Run the LLMChain for the second prompt
response_fact = chain_fact.run(input_data)

print(f"Genres: {genre1}, {genre2}, {genre3}")
print(f"Fact: {response_fact}")

Genres: jazz, pop, rock
Fact: 
Jazz, pop, and rock are three popular genres of music that have evolved over time and have a wide range of styles and influences. They all have distinct characteristics and elements that make them unique, such as improvisation in jazz, catchy melodies in pop, and heavy guitar riffs in rock. These genres have also influenced and been influenced by each other, leading to the creation of sub-genres and fusion styles. They have a strong presence in mainstream music and have been embraced by audiences of all ages and backgrounds.


In this example, the second prompt is constructed poorly. It asks to "tell me something about {genre1}, {genre2}, and {genre3} without giving any specific details." This prompt is unclear, as it asks for information about the genres but also states not to provide specific details. This makes it difficult for the LLM to generate a coherent and informative response. As a result, the LLM may provide a less informative or confusing answer.

The first prompt asks for "some musical genres" **without specifying any criteria or context**, and the second prompt asks why the given genres are "unique" **without providing any guidance** on what aspects of uniqueness to focus on, such as their historical origins, stylistic features, or cultural significance.

![rainbow](https://github.com/ancilcleetus/My-Learning-Journey/assets/25684256/839c3524-2a1d-4779-85a0-83c562e1e5e5)

# 8. Chain of Thought Prompting

Chain of Thought Prompting (CoT) is a technique developed to encourage large language models to explain their reasoning process, leading to more accurate results. By providing few-shot exemplars demonstrating the reasoning process, the LLM is guided to explain its reasoning when answering the prompt. This approach has been found effective in improving results on tasks like arithmetic, common sense, and symbolic reasoning.

In the context of LangChain, CoT can be beneficial for several reasons. First, it can help break down complex tasks by assisting the LLM in decomposing a complex task into simpler steps, making it easier to understand and solve the problem. This is particularly useful for calculations, logic, or multi-step reasoning tasks. Second, CoT can guide the model through related prompts, helping generate more coherent and contextually relevant outputs. This can lead to more accurate and useful responses in tasks that require a deep understanding of the problem or domain.

There are some limitations to consider when using CoT. One limitation is that it has been found to yield performance gains only when used with models of approximately 100 billion parameters or larger; smaller models tend to produce illogical chains of thought, which can lead to worse accuracy than standard prompting. Another limitation is that CoT may not be equally effective for all tasks. It has been shown to be most effective for tasks involving arithmetic, common sense, and symbolic reasoning. For other types of tasks, the benefits of using CoT might be less pronounced or even counterproductive.

![rainbow](https://github.com/ancilcleetus/My-Learning-Journey/assets/25684256/839c3524-2a1d-4779-85a0-83c562e1e5e5)

# 9. Tips for Effective Prompt Engineering

- **Be specific** with your prompt: Provide enough context and detail to guide the LLM toward the desired output.
- **Force conciseness** when needed.
- **Encourage the model to explain its reasoning:** This can lead to more accurate results, especially for complex tasks.

Keep in mind that prompt engineering is an iterative process, and it may require several refinements to obtain the best possible answer. As LLMs become more integrated into products and services, the ability to create effective prompts will be an important skill to have.

A well-structured prompt example is given below:

In [7]:
from langchain import PromptTemplate, FewShotPromptTemplate, LLMChain
from langchain.llms import OpenAI

# Initialize LLM
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)

examples = [
    {
        "query": "What's the secret to happiness?",
        "answer": "Finding balance in life and learning to enjoy the small moments."
    }, {
        "query": "How can I become more productive?",
        "answer": "Try prioritizing tasks, setting goals, and maintaining a healthy work-life balance."
    }
]

example_template = """
User: {query}
AI: {answer}
"""

example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

prefix = """The following are excerpts from conversations with an AI
life coach. The assistant provides insightful and practical advice to the users' questions. Here are some
examples:
"""

suffix = """
User: {query}
AI: """

few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

# Create the LLMChain for the few-shot prompt template
chain = LLMChain(llm=llm, prompt=few_shot_prompt_template)

# Define the user query
user_query = "What are some tips for improving communication skills?"

# Run the LLMChain for the user query
response = chain.run({"query": user_query})

print(f"User Query: {user_query}")
print(f"AI Response: {response}")

User Query: What are some tips for improving communication skills?
AI Response: Active listening, being clear and concise, and practicing empathy are all important skills for effective communication. Also, don't be afraid to ask for clarification or feedback to improve your communication.


This prompt:

- **Provides a clear context in the prefix:** The prompt states that the AI is a life coach providing insightful and practical advice. This context helps guide the AI's responses and ensures they align with the intended purpose.

- **Uses examples that demonstrate the AI's role and the type of responses it generates:** By providing relevant examples, the AI can better understand the style and tone of the responses it should produce. These examples serve as a reference for the AI to generate similar responses that are consistent with the given context.

- **Separates examples and the actual query:** This allows the AI to understand the format it should follow, ensuring a clear distinction between example conversations and the user's input. This separation helps the AI to focus on the current query and respond accordingly.

- **Includes a clear suffix that indicates where the user's input goes and where the AI should provide its response:** The suffix acts as a cue for the AI, showing where the user's query ends and the AI's response should begin. This structure helps maintain a clear and consistent format for the generated responses.

By using this well-structured prompt, the AI can understand its role, the context, and the expected response format, leading to more accurate and useful outputs.

![rainbow](https://github.com/ancilcleetus/My-Learning-Journey/assets/25684256/839c3524-2a1d-4779-85a0-83c562e1e5e5)